In [5]:
from pathlib import Path

import pandas as pd
import pydash as _
import numpy as np

In [6]:
df_answer = pd.read_csv(Path('./answers.csv'))
df_answer

,_index,question_id,correct_answer_a,correct_answer_b
0,1,pitch-random-single-0-q1,6,NaN
1,2,pitch-random-single-1-q1,4,NaN
2,3,pitch-random-single-2-q1,4,NaN
3,4,pitch-random-single-3-q1,8,NaN
4,5,pitch-random-double-0-q1,4,-4
...,...,...,...,...
67,68,tempo-linear-double-1-q2,c,c
68,69,tempo-linear-double-2-q1,d,c
69,70,tempo-linear-double-2-q2,a,c
70,71,tempo-linear-double-3-q1,b,c


In [7]:
# write a function to check if the input is a numeric value
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


In [8]:
answers = []
counter = 0

for r in df_answer.to_dict(orient='records'):
    condition, trend_type, series_type, dataset_index, question_index = r['question_id'].split('-')
    # print(condition, trend_type, series_type, dataset_index, question_index)
    answers.append({
        'question_id': "-".join(r['question_id'].split('-')[1:3])+"-a",
        "question_type":f"{r['question_id']}-a",
        "condition": condition,
        "_index":counter,
        'trend_type': trend_type,
        'series_type': series_type,
        'dataset_index': dataset_index,
        'question_index': question_index,
        'question_sequence:': 'a',
        'answer_type': 'numeric' if is_numeric(r['correct_answer_a']) else 'categorical',
        'answer': r['correct_answer_a']
    })
    if not pd.isna(r['correct_answer_b']):
        counter += 1
        answers.append({
            'question_id': "-".join(r['question_id'].split('-')[1:3])+"-b",
            "condition": condition,
            "question_type":f"{r['question_id']}-b",
            'trend_type': trend_type,
            "_index":counter,
            'series_type': series_type,
            'dataset_index': dataset_index,
            'question_index': question_index,
            'question_sequence:': 'b',
            'answer_type': 'numeric' if is_numeric(r['correct_answer_a']) else 'categorical',
            'answer': r['correct_answer_b']
        })
    counter += 1

df_answers = pd.DataFrame(answers)
df_answers

,question_id,question_type,condition,_index,trend_type,series_type,dataset_index,question_index,question_sequence:,answer_type,answer
0,random-single-a,pitch-random-single-0-q1-a,pitch,0,random,single,0,q1,a,numeric,6
1,random-single-a,pitch-random-single-1-q1-a,pitch,1,random,single,1,q1,a,numeric,4
2,random-single-a,pitch-random-single-2-q1-a,pitch,2,random,single,2,q1,a,numeric,4
3,random-single-a,pitch-random-single-3-q1-a,pitch,3,random,single,3,q1,a,numeric,8
4,random-double-a,pitch-random-double-0-q1-a,pitch,4,random,double,0,q1,a,numeric,4
...,...,...,...,...,...,...,...,...,...,...,...
103,linear-double-b,tempo-linear-double-2-q2-b,tempo,103,linear,double,2,q2,b,categorical,c
104,linear-double-a,tempo-linear-double-3-q1-a,tempo,104,linear,double,3,q1,a,categorical,b
105,linear-double-b,tempo-linear-double-3-q1-b,tempo,105,linear,double,3,q1,b,categorical,c
106,linear-double-a,tempo-linear-double-3-q2-a,tempo,106,linear,double,3,q2,a,categorical,c


In [9]:
# df_answers[df_answers['question_id'] == 'tempo-random-single-0-q1-a']['answer'].values[0]

df_answers.head()


,question_id,question_type,condition,_index,trend_type,series_type,dataset_index,question_index,question_sequence:,answer_type,answer
0,random-single-a,pitch-random-single-0-q1-a,pitch,0,random,single,0,q1,a,numeric,6
1,random-single-a,pitch-random-single-1-q1-a,pitch,1,random,single,1,q1,a,numeric,4
2,random-single-a,pitch-random-single-2-q1-a,pitch,2,random,single,2,q1,a,numeric,4
3,random-single-a,pitch-random-single-3-q1-a,pitch,3,random,single,3,q1,a,numeric,8
4,random-double-a,pitch-random-double-0-q1-a,pitch,4,random,double,0,q1,a,numeric,4


In [10]:
def normalize_answer(df):
    results = []
    temp_counter =0
    for r in df.to_dict(orient='records'):
        
        results.append({
            'question_type': f"{r['question_id']}-a",
            "_index": temp_counter,
            'response': r['answer'],
        })
        if not pd.isna(r['answer_2']):
            temp_counter += 1
            results.append({
                'question_type': f"{r['question_id']}-b",
                "_index": temp_counter,
                'response': r['answer_2'],
            })
            
        temp_counter += 1
    return pd.DataFrame(results)

def calculate_diff(r):
    question = df_answers[df_answers['_index'] == r['_index']]
    ans = question['answer'].values[0]
    ans_type = question['answer_type'].values[0]
    if ans_type == 'numeric':
        return abs(int(ans) - int(r['response']))
    else:
        return ans == r['response']

def calculate_sign(r):
    question = df_answers[df_answers['_index'] == r['_index']]
    ans = question['answer'].values[0]
    ans_type = question['answer_type'].values[0]
    if ans_type == 'numeric':
        return (float(ans)> 0 and  float(r['response'])>0 )  or (float(ans) < 0 and float(r['response']) < 0)
        # return a nan value
        return False


results_dfs = []
exp_n = 0
for f in Path('./exp/').iterdir():
    exp_n += 1
    exp_id = f.stem
    df_exp_result = pd.read_csv(f)
    df_exp_result = normalize_answer(df_exp_result)
    df_exp_result[f'diff_{exp_id}'] = df_exp_result.apply(calculate_diff, axis=1)
    df_exp_result[f'sign_{exp_id}'] = df_exp_result.apply(calculate_sign, axis=1)
    df_exp_result = df_exp_result.rename(columns={'response': f"response_{exp_id}"})
    df_exp_result = df_exp_result.rename(columns={'_index': f"index_{exp_id}"})

    results_dfs.append(df_exp_result)



In [11]:
results_dfs[0]

,question_type,index_5,response_5,diff_5,sign_5
0,tempo-random-single-0-q1-a,0,-6,12,False
1,tempo-random-single-1-q1-a,1,-2,6,False
2,tempo-random-single-2-q1-a,2,-2,6,False
3,tempo-random-single-3-q1-a,3,3,5,True
4,tempo-random-double-0-q1-a,4,-5,9,False
...,...,...,...,...,...
103,spatial-linear-double-2-q2-b,103,c,True,None
104,spatial-linear-double-3-q1-a,104,b,True,None
105,spatial-linear-double-3-q1-b,105,c,True,None
106,spatial-linear-double-3-q2-a,106,c,True,None


In [12]:
df_results = results_dfs[0]


for df in results_dfs[1:-1]:
    df_results = pd.merge(df_results, df, on='question_type')


In [13]:
df_results

,question_type,index_5,response_5,diff_5,sign_5,index_1,response_1,diff_1,sign_1,index_2,response_2,diff_2,sign_2,index_6,response_6,diff_6,sign_6
0,tempo-random-single-0-q1-a,0,-6,12,False,72,-5,2,True,36,4,5,False,0,8,2,True
1,tempo-random-single-1-q1-a,1,-2,6,False,73,-3,0,True,37,4,6,False,1,7,3,True
2,tempo-random-single-2-q1-a,2,-2,6,False,74,-1,4,True,38,2,6,False,2,8,4,True
3,tempo-random-single-3-q1-a,3,3,5,True,75,5,1,True,39,8,6,True,3,10,2,True
4,tempo-random-double-0-q1-a,4,-5,9,False,76,-5,2,True,40,9,3,True,4,9,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,spatial-linear-double-2-q2-b,103,c,True,None,67,c,True,None,103,c,True,None,67,c,True,None
104,spatial-linear-double-3-q1-a,104,b,True,None,68,a,False,None,104,c,False,None,68,a,False,None
105,spatial-linear-double-3-q1-b,105,c,True,None,69,a,False,None,105,b,False,None,69,a,False,None
106,spatial-linear-double-3-q2-a,106,c,True,None,70,b,False,None,106,c,True,None,70,c,False,None


In [14]:
df_answers

,question_id,question_type,condition,_index,trend_type,series_type,dataset_index,question_index,question_sequence:,answer_type,answer
0,random-single-a,pitch-random-single-0-q1-a,pitch,0,random,single,0,q1,a,numeric,6
1,random-single-a,pitch-random-single-1-q1-a,pitch,1,random,single,1,q1,a,numeric,4
2,random-single-a,pitch-random-single-2-q1-a,pitch,2,random,single,2,q1,a,numeric,4
3,random-single-a,pitch-random-single-3-q1-a,pitch,3,random,single,3,q1,a,numeric,8
4,random-double-a,pitch-random-double-0-q1-a,pitch,4,random,double,0,q1,a,numeric,4
...,...,...,...,...,...,...,...,...,...,...,...
103,linear-double-b,tempo-linear-double-2-q2-b,tempo,103,linear,double,2,q2,b,categorical,c
104,linear-double-a,tempo-linear-double-3-q1-a,tempo,104,linear,double,3,q1,a,categorical,b
105,linear-double-b,tempo-linear-double-3-q1-b,tempo,105,linear,double,3,q1,b,categorical,c
106,linear-double-a,tempo-linear-double-3-q2-a,tempo,106,linear,double,3,q2,a,categorical,c


In [15]:
df_filtered_results = df_results.filter(regex='question|diff')
df_filtered_results.head()

,question_type,diff_5,diff_1,diff_2,diff_6
0,tempo-random-single-0-q1-a,12,2,5,2
1,tempo-random-single-1-q1-a,6,0,6,3
2,tempo-random-single-2-q1-a,6,4,6,4
3,tempo-random-single-3-q1-a,5,1,6,2
4,tempo-random-double-0-q1-a,9,2,3,5


In [16]:
# try to merge with the big table of answers

df_merged = df_answers.merge(df_filtered_results, on='question_type')
df_merged

# melt the dataframe to make it easier to work with
df_melted = pd.melt(df_merged, id_vars=df_merged.columns[:11], value_vars=df_merged.columns[11:])


In [17]:
df_melted["value"]

0          3
1          7
2          7
3          4
4          1
       ...  
427     True
428    False
429    False
430     True
431     True
Name: value, Length: 432, dtype: object

In [18]:
for qid in df_melted['condition'].unique():

    df = df_melted[df_melted['condition'] == qid]
    df = df[df['answer_type'] == 'numeric']
    df = df[df['series_type'] == 'single']
    print(qid, df['value'].mean(), df['value'].std(), df['value'].min(), df['value'].max(), df['value'].max() - df['value'].min())

#df_melted.groupby('question_id').agg(['mean', 'std', 'min', 'max']).to_csv('./data_analysis.csv')
# df_answer_diff_melted.groupby('question_id').agg(['mean', 'std', 'min', 'max'])

pitch 4.0625 3.5302266216207707 0 11 11
spatial 1.8125 1.2230426539304888 0 4 4
tempo 4.375 2.84897642437888 0 12 12


In [19]:
for qid in df_melted['condition'].unique():

    df = df_melted[df_melted['condition'] == qid]
    df = df[df['answer_type'] == 'numeric']
    df = df[df['series_type'] == 'double']
    print(qid, df['value'].mean(), df['value'].std(), df['value'].min(), df['value'].max(), df['value'].max() - df['value'].min())


pitch 4.28125 4.753500408018523 0 19 19
spatial 3.09375 3.4111711639373095 0 13 13
tempo 4.0 3.8854567548163996 0 15 15


In [20]:
for qid in df_melted['condition'].unique():

    df = df_melted[df_melted['condition'] == qid]
    df = df[df['answer_type'] == 'numeric']
    print(qid, df['value'].mean(), df['value'].std(), df['value'].min(), df['value'].max(), df['value'].max() - df['value'].min())

pitch 4.208333333333333 4.3464748902566095 0 19 19
spatial 2.6666666666666665 2.919729901041441 0 13 13
tempo 4.125 3.5467995762437092 0 15 15


In [47]:
for qid in df_melted['condition'].unique():

    df = df_melted[df_melted['condition'] == qid]
    df = df[df['answer_type'] == 'numeric']
    print(qid, df['value'].mean(), df['value'].std(), df['value'].min(), df['value'].max(), df['value'].max() - df['value'].min())

pitch 4.208333333333333 4.3464748902566095 0 19 19
spatial 2.6666666666666665 2.919729901041441 0 13 13
tempo 4.125 3.5467995762437092 0 15 15


In [64]:
df_numerical_only = df_melted[df_melted['answer_type'] == 'numeric']
df_numerical_only = df_numerical_only[df_numerical_only['series_type'] == 'double']

type1 = df_numerical_only[df_numerical_only['condition']=='pitch']
type2 = df_numerical_only[df_numerical_only['condition']=='spatial']
type3 = df_numerical_only[df_numerical_only['condition']=='tempo']

In [68]:
# change type1 from tuple to dict
type1_dic = type1.to_dict(orient='records')
type2_dic = type2.to_dict(orient='records')
type3_dic = type3.to_dict(orient='records')

# perform the t-test
ttest_ind([x['value'] for x in type1_dic], [x['value'] for x in type2_dic])


Ttest_indResult(statistic=1.1481361506943524, pvalue=0.25532306699759993)

In [59]:
# perform one way anova
f_oneway([x['value'] for x in type1_dic], [x['value'] for x in type2_dic], [x['value'] for x in type3_dic])

F_onewayResult(statistic=2.7064899370511575, pvalue=0.07024041558267663)

In [58]:
from scipy.stats import ttest_ind

#ttest_ind(df_pitch['value'].to_list(),df_spatial['value'].to_list(),df_tempo['value'].to_list())

# do a repeated measures anova
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm


F_onewayResult(statistic=-3.1317518808867945e-31, pvalue=nan)